In [6]:
import glob
import numpy as np
import tensorflow as tf
from scipy.io import wavfile

In [7]:
clean_wav_files = glob.glob('./data/clean_trainset_28spk_wav/*.wav')
noisy_wav_files = glob.glob('./data/noisy_trainset_28spk_wav/*.wav')

clean_data = [wavfile.read(file)[1] for file in clean_wav_files]
noisy_data = [wavfile.read(file)[1] for file in noisy_wav_files]
frame_rate = wavfile.read(clean_wav_files[0])[0]

In [8]:
sequence_length = int(frame_rate*40/1000) # 40 miliseconds time windows
sequence_shift = int(frame_rate*16/1000)  # 16 milisceonds shifts
print(frame_rate, sequence_length, sequence_shift)

48000 1920 768


In [9]:
def window(noisy_data: np.array, clean_data: np.array, shift, length):
    """[summary]

    Args:
        noisy_data (np.array): [description]
        clean_data (np.array): [description]

    Returns:
        [type]: [description]
    """
    x = []
    y = []
    for i in range(len(noisy_data)):
        for j in range((len(noisy_data[i]) - length)//shift + 1):
            x.append(noisy_data[i][j*shift: j*shift + length])
            y.append(clean_data[i][j*shift: j*shift + length])
        if i==2: # remove this if to get the whole dataset
            break
    x = np.expand_dims(np.array(x), axis=0)
    y = np.expand_dims(np.array(y), axis=0)
    return x, y

In [10]:
x_train, y_train = window(noisy_data, clean_data, shift=sequence_shift, length=sequence_length)
print(x_train.shape)

(1, 416, 1920)


In [39]:
def GRU(sequence_length, neurons=40):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, GRU
    
    model = Sequential()
    model.add(GRU(neurons, return_sequences=False, return_state=False, input_shape=(1, sequence_length)))
    model.add(Dense(sequence_length, activation='relu'))
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['acc'])
    
    return model

In [40]:
model = GRU(sequence_length)
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_10 (GRU)                 (None, 40)                235440    
_________________________________________________________________
dense_8 (Dense)              (None, 1920)              78720     
Total params: 314,160
Trainable params: 314,160
Non-trainable params: 0
_________________________________________________________________


In [41]:
hist = model.fit(x=x_train, y=y_train, epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 644us/step - loss: 5500875.0000 - acc: 0.0024
Epoch 2/100
1/1 [==============================] - 0s 606us/step - loss: 5500871.5000 - acc: 0.0024
Epoch 3/100
1/1 [==============================] - 0s 642us/step - loss: 5500868.5000 - acc: 0.0024
Epoch 4/100
1/1 [==============================] - 0s 709us/step - loss: 5500865.0000 - acc: 0.0024
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss: 5500863.0000 - acc: 0.0024
Epoch 6/100
1/1 [==============================] - 0s 642us/step - loss: 5500860.5000 - acc: 0.0024
Epoch 7/100
1/1 [==============================] - 0s 631us/step - loss: 5500858.0000 - acc: 0.0024
Epoch 8/100
1/1 [==============================] - 0s 682us/step - loss: 5500855.5000 - acc: 0.0024
Epoch 9/100
1/1 [==============================] - 0s 634us/step - loss: 5500854.0000 - acc: 0.0024
Epoch 10/100
1/1 [==============================] - 0s 630us/step - loss: 5500851.5000 - acc: 0.0024
E